# Plane-based sampling
- Idea:

Approach 1:
    - subsample using the radius again, construct a plane based on the centroid and 2 closest points -> deviation from this plane approximates curvature

Approach 2:
    - DO PCA to find normal vector

A[


- Flaws:
    - depends on radius chosen


In [5]:
import utils
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

In [17]:
train_dataset = utils.import_train(100_00)

In [18]:
train_dataset

ModelNet10(3991)

In [19]:
point_cloud = train_dataset[1]  # 1000 = chair

In [20]:
utils.draw_pointcloud(point_cloud)

Visualisation of neighbourhood function

In [21]:
point0 = point_cloud.pos[1]
nr_points = len(point_cloud.pos)
radius = .1
neighbours0 = utils.find_neighborhood(point_cloud.pos, point0, radius)

In [22]:
utils.draw_pointcloud_neighbour(point_cloud.pos, neighbours0, point0,radius,False)

**Find KNN**

- later maybe want homogenous neighbourhood instead

In [45]:
def k_nearest_neighbors_single(point_cloud, query_point, k):
    # Calculate squared Euclidean distances between the query point and all points in the point cloud
    distances = torch.linalg.vector_norm(point_cloud - query_point,dim=1, ord=2)

    # Find the indices of the k nearest neighbors
    _, indices = torch.topk(distances, k=k, largest=False)

    # Retrieve the k nearest neighbors from the point cloud
    nearest_neighbors = point_cloud[indices]

    return nearest_neighbors

In [98]:
knn0 = k_nearest_neighbors_single(point_cloud.pos, point0,3) # includes point0 in res
knn0

tensor([[-0.7924,  0.8363, -0.3700],
        [-0.7879,  0.8362, -0.3700],
        [-0.7868,  0.8208, -0.3700]])

In [51]:
%matplotlib qt
x = point_cloud.pos[:, 0]
y = point_cloud.pos[:, 1]
z = point_cloud.pos[:, 2]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(x, y, z, marker='.', alpha=.01)
ax.scatter(point0[0],point0[1],point0[2], color="green")
ax.scatter(knn0[:,0],knn0[:,1],knn0[:,2], color='red')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=30, azim=270)


**Construct plane from 2 neighbours**

In [116]:
import numpy as np

def fit_plane(points):
    # Formulate the data matrix
    data_matrix = np.array(points)

    # Center the data
    centered_data = data_matrix - np.mean(data_matrix, axis=1)

    # Perform PCA
    _, _, V = np.linalg.svd(centered_data)

    # Extract the eigenvectors
    normal_vector = V[-1]

    return normal_vector

# Example points in 3D space
points = [[1, 2, 3],
          [4, 5, 6],
          [7, 8, 9]]

# Fit a plane through the points using PCA
normal_vector = fit_plane(points)

print("Normal vector of the plane:", normal_vector)


Normal vector of the plane: [-0.40824829  0.81649658 -0.40824829]


In [117]:
normal_vector = fit_plane(knn0)


In [125]:
np_knn = np.array(knn0)

In [248]:
u, sigma, v = np.linalg.svd(np_knn - np.mean(np_knn, axis=1, keepdims=True))

# Extract the left singular vectors
normal = v[:,1]
normal

array([-0.79075384, -0.20341235, -0.5773489 ], dtype=float32)

In [249]:
normal


array([-0.79075384, -0.20341235, -0.5773489 ], dtype=float32)

In [250]:
centroid = np.array(point0)
centroid

array([-0.7924145 ,  0.83634365, -0.37003624], dtype=float32)

In [251]:
x_bound = (-0.7926955819129944, -0.7865114808082581)
y_bound = (0.8200439155101776, 0.8371198236942291)

In [252]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(knn0[:,0], knn0[:,1], knn0[:,2], marker='o', alpha=1)
xx, yy = np.meshgrid(np.linspace(x_bound[0],x_bound[1],num=10), np.linspace(y_bound[0],y_bound[1],num=10))
d = normal[0] * centroid[0] + normal[1] * centroid[1] + normal[2] * centroid[2]
z = (-normal[0] * xx - normal[1] * yy + d) * 1. /normal[2]
# Visualize the points and the plane
plane = ax.plot_surface(xx, yy, z, alpha=0.3)
# plane = ax.plot_surface(xx, yy, zz, alpha=0.3)
ax.set_xlim(x_bound)
ax.set_ylim(y_bound)
ax.set_zlim((-.4,-.35))

(-0.4, -0.35)

In [ ]:
################

tensor([-0.2491, -0.3700, -0.2491,  ...,  0.0818, -0.0396, -0.3700])

In [15]:
raise KeyboardInterrupt

KeyboardInterrupt: 

# NOTE: should be weighted average distance to plane as closer points are more relevant to the curvature around that point

In [ ]:
def calculate_average_distance(center_point, surrounding_points):
    # Calculate Euclidean distances from the center point to all surrounding points
    distances = np.linalg.norm(surrounding_points - center_point, axis=1)

    # Sort distances and select the indices of the 2 closest points
    closest_indices = np.argsort(distances)[:3]

    # Select the 2 closest points
    closest_points = surrounding_points[closest_indices]

    # Construct the plane using the three closest points
    v1 = closest_points[1] - closest_points[0]
    v2 = closest_points[2] - closest_points[0]
    normal_vector = np.cross(v1, v2)
    print(closest_points)
    print(normal_vector)
    d = -np.dot(normal_vector, closest_points[0])

    # Calculate the average distance from the plane for all surrounding points
    distances_from_plane = np.abs(np.dot(surrounding_points, normal_vector) + d) / np.linalg.norm(normal_vector)
    average_distance = np.mean(distances_from_plane)

    return average_distance


In [ ]:
distances = np.linalg.norm(neighbours0 - point0, axis=1)

# Sort distances and select the indices of the three closest points
closest_indices = np.argsort(distances)[:5]

# Select the 2 closest points
closest_points = neighbours0[closest_indices]

In [ ]:
res = np.cross(closest_points[1], closest_points[0])
x_range = np.linspace(closest_points[0][0] - radius, closest_points[0][1] + radius, 10)
y_range = np.linspace(closest_points[0][1] - radius, closest_points[0][1] + radius, 10)
x_plane, y_plane = np.meshgrid(x_range, y_range)

z_plane = (-res[0] * x_plane - res[1] * y_plane)

In [ ]:
print(closest_points)
v1 = closest_points[2] - closest_points[0]
v2 = closest_points[1] - closest_points[0]

print(normal_vector)

In [ ]:
def visualize_plane(center_point, surrounding_points,point_cloud, radius, limit_axis=False,elev=30,azim=100):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Scatter plot for surrounding points
    ax.scatter(surrounding_points[:, 0], surrounding_points[:, 1], surrounding_points[:, 2], c='orange', label='Surrounding Points', alpha=0.1)

    # Scatter plot for center point
    ax.scatter(center_point[0], center_point[1], center_point[2], c='r', marker='o', label='Center Point',s=100)

    # Scatter plot for point_cloud
    x = point_cloud.pos[:, 0]
    y = point_cloud.pos[:, 1]
    z = point_cloud.pos[:, 2]

    ax.scatter(x, y, z, marker='.', alpha=.01)

    # Plane
    distances = np.linalg.norm(surrounding_points - center_point, axis=1)

    # Sort distances and select the indices of the 2 closest points
    closest_indices = np.argsort(distances)[:3]

    # Select the 2 closest points
    closest_points = surrounding_points[closest_indices]

    # Construct the plane using the three closest points
    v1 = closest_points[1] - closest_points[0]
    v2 = closest_points[2] - closest_points[0]
    normal_vector = np.cross(v1, v2)
    d = -np.dot(normal_vector, surrounding_points[0])
    x_range = np.linspace(center_point[0]-radius, center_point[1]+radius, 10)
    y_range = np.linspace(center_point[1]-radius, center_point[1]+radius, 10)
    x_plane, y_plane = np.meshgrid(x_range, y_range)
    z_plane = (-normal_vector[0] * x_plane - normal_vector[1] * y_plane - d)

    # Plot the plane
    ax.plot_surface(x_plane, y_plane, z_plane, alpha=0.3, color='g', label='Plane')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    # legend
    scatter_proxy = plt.Line2D([0], [0], linestyle='none', c='b', marker='o')
    plane_proxy = plt.Line2D([0], [0], linestyle='-', c='g')
    center_proxy = plt.Line2D([0], [0], linestyle='none', c='r', marker='o')
    ax.legend([scatter_proxy, plane_proxy, center_proxy], ['Surrounding Points', 'Plane', 'Center Point'])
    ax.view_init(elev=elev, azim=azim)
    x_min, x_max = center_point[0] - radius, center_point[0] + radius
    y_min, y_max = center_point[1] - radius, center_point[1] + radius
    z_min, z_max = center_point[2] - radius, center_point[2] + radius
    if limit_axis:
        ax.set_xlim([x_min, x_max])
        ax.set_ylim([y_min, y_max])
        ax.set_zlim([z_min, z_max])

In [ ]:
%matplotlib qt
visualize_plane(point0, neighbours0,point_cloud, radius,True)

In [ ]:
calculate_average_distance(point0,neighbours0)

In [ ]:
v1 = neighbours0[1] - neighbours0[0]
v2 = neighbours0[2] - neighbours0[0]
normal_vector = np.cross(v1, v2)
d = -np.dot(normal_vector, neighbours0[0])
x_range = np.linspace(neighbours0[0]-radius, point0[1]+radius, 10)
y_range = np.linspace(neighbours0[1]-radius, point0[1]+radius, 10)
x_plane, y_plane = np.meshgrid(x_range, y_range)
z_plane = (-normal_vector[0] * x_plane - normal_vector[1] * y_plane - d) / normal_vector[2]


In [ ]:
print(normal_vector)

In [ ]:
utils.visualize_plane(point0, neighbours0,point_cloud, radius,True)